In [ ]:
import sys
import os
import functools
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import wandb
import torch
from src.utils import load_env_vars
from src.dataset import Cifar10DataManager
from src.training import run_training_sweep
import json

# Setup
env = load_env_vars()
PROJECT_NAME = env.get("WANDB_PROJECT", "cifar10_mlops_project")
ENTITY = env.get("WANDB_ENTITY", None)

print(f"Project: {PROJECT_NAME}, Entity: {ENTITY}")

# 1. Download Dataset from W&B (Ensure consistency)
# We initialize a run just to fetch the data
run = wandb.init(project=PROJECT_NAME, job_type="training_prep")
artifact = run.use_artifact(f'{ENTITY}/{PROJECT_NAME}/cifar10_dataset:latest', type='dataset')
artifact_dir = artifact.download(root="../data")
run.finish()

print(f"Data downloaded to {artifact_dir}")

# 2. Define Sweep Config
sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'val_acc', 'goal': 'maximize'},
    'parameters': {
        'learning_rate': {'min': 0.001, 'max': 0.1},
        'batch_size': {'values': [64, 128]},
        'optimizer': {'values': ['adam', 'sgd']},
        'architecture_option': {'values': ['standard', 'modified']},
        'epochs': {'value': 5}
    }
}

# 3. Initialize Sweep
sweep_id = wandb.sweep(sweep_config, project=PROJECT_NAME)
print(f"Sweep ID: {sweep_id}")

# Save Sweep ID for next steps
os.makedirs("../artifacts", exist_ok=True)
with open("../artifacts/sweep_id.txt", "w") as f:
    f.write(sweep_id)

# 4. Run Agent
# We use partial to pass the correct data_dir to the training function
train_func = functools.partial(run_training_sweep, data_dir="../data")
wandb.agent(sweep_id, train_func, count=5)

# 5. Extract Best Config
api = wandb.Api()
sweep = api.sweep(f"{ENTITY}/{PROJECT_NAME}/{sweep_id}")
best_run = sweep.best_run()
best_config = best_run.config
print(f"Best Run: {best_run.name} ({best_run.id}) with Val Acc: {best_run.summary.get('val_acc')}")

# Save best config for later Steps (Deployment & Retraining)
with open("../artifacts/best_config.json", "w") as f:
    json.dump(best_config, f)
    
print("Step 2 Complete: Sweep finished and best config saved.")

# Model Training and Hyperparameter Sweep

This notebook implements the training pipeline with Transfer Learning (ResNet18) and uses W&B Sweeps for hyperparameter optimization.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import wandb
import os
import copy
from pathlib import Path

# setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Define Sweep Configuration
sweep_config = {
    'method': 'bayes', # Bayesian optimization
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    'parameters': {
        'learning_rate': {
            'min': 0.0001,
            'max': 0.1
        },
        'batch_size': {
            'values': [32, 64, 128]
        },
        'optimizer': {
            'values': ['adam', 'sgd']
        },
        'architecture_option': {
            # standard: The way we used here (Option C)
            # upsample: Upsampling to 224x224 (Option A - Lazy Way)
            # modified: Modify first layer (Option B - Pro Way)
            'values': ['standard', 'upsample', 'modified']
        },
        'epochs': {
            'value': 20 # Keep it small for demonstration, increase for real results
        }
    }
}

PROJECT_NAME = "cifar10_mlops_project"
sweep_id = wandb.sweep(sweep_config, project=PROJECT_NAME)

# --- AUTOMATION: SAVE SWEEP ID FOR NEXT STEPS ---
os.makedirs("../artifacts", exist_ok=True)
with open("../artifacts/sweep_id.txt", "w") as f:
    f.write(sweep_id)
print(f"Sweep ID {sweep_id} saved to ../artifacts/sweep_id.txt")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: b8ld9rmu
Sweep URL: https://wandb.ai/esi-sba-dz/cifar10_mlops_project/sweeps/b8ld9rmu


In [ ]:
def build_dataset(batch_size, architecture_option):
    # Base transforms list
    transforms_list_train = [
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
    
    transforms_list_test = [
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]

    # Adjust transforms based on architecture option
    if architecture_option == 'upsample':
        # Option A: Lazy Way (Resize to 224x224)
        transforms_list_train.insert(0, transforms.Resize(224))
        transforms_list_test.insert(0, transforms.Resize(224))
    else:
        # Option B (Modified) and C (Standard): Keep 32x32, usually with padding/cropping for train
        transforms_list_train.insert(0, transforms.RandomCrop(32, padding=4))

    transform_train = transforms.Compose(transforms_list_train)
    transform_test = transforms.Compose(transforms_list_test)

    trainset = torchvision.datasets.CIFAR10(root='../data/raw', train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

    testset = torchvision.datasets.CIFAR10(root='../data/raw', train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    return trainloader, testloader

def build_model(architecture_option):
    # Load pretrained ResNet18
    model = torchvision.models.resnet18(pretrained=True)
    
    if architecture_option == 'modified':
        # Option B: The "Pro" Way
        # Replace 7x7 conv with 3x3 conv, stride 1, padding 1
        model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        # Remove MaxPool
        model.maxpool = nn.Identity()
    
    # Note: Option A (Upsample) and C (Standard) use the unmodified backbone structure
    # Standard ResNet on 32x32 (Option C) is suboptimal but works.

    # Replace last layer for CIFAR-10 (10 classes)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 10)
    
    return model.to(device)

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        config = wandb.config
        
        # Access the architecture option from config (with default if missing)
        arch_opt = getattr(config, 'architecture_option', 'standard')
        
        trainloader, testloader = build_dataset(config.batch_size, arch_opt)
        model = build_model(arch_opt)
        
        criterion = nn.CrossEntropyLoss()
        if config.optimizer == "sgd":
            optimizer = optim.SGD(model.parameters(), lr=config.learning_rate, momentum=0.9)
        elif config.optimizer == "adam":
            optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
            
        best_acc = 0.0
        
        print(f"Starting training with option: {arch_opt}, lr: {config.learning_rate}, batch: {config.batch_size}")
        
        for epoch in range(config.epochs):
            model.train()
            running_loss = 0.0
            
            for i, (inputs, labels) in enumerate(trainloader):
                inputs, labels = inputs.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
                
                # Log batch metrics
                wandb.log({"batch_loss": loss.item()})
            
            # Validation
            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            
            val_acc = 100 * correct / total
            epoch_loss = running_loss / len(trainloader)
            
            # Log epoch metrics
            wandb.log({"epoch": epoch, "loss": epoch_loss, "val_acc": val_acc})
            print(f"Epoch {epoch}: Loss {epoch_loss:.3f}, Val Acc {val_acc:.2f}%")
            
            # Save best model to W&B
            if val_acc > best_acc:
                best_acc = val_acc
                
                # Create models directory
                Path("../models").mkdir(parents=True, exist_ok=True)
                
                # Save locally
                model_path = f"../models/model_best_{wandb.run.id}.pth"
                torch.save(model.state_dict(), model_path)
                
                # Log as artifact
                artifact = wandb.Artifact(f"model-best-{wandb.run.id}", type="model")
                artifact.add_file(model_path)
                wandb.log_artifact(artifact)
                print(f"New best model saved with acc: {best_acc}")

In [ ]:
# Run the Sweep Agent
# count=5 means run 5 experiments
print(f"Starting sweep agent for ID: {sweep_id}")
wandb.agent(sweep_id, train, count=6)

wandb: Agent Starting Run: dd3hx67d with config:
wandb: 	batch_size: 128
wandb: 	epochs: 5
wandb: 	learning_rate: 0.027101044512480844
wandb: 	optimizer: sgd
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.
wandb: Currently logged in as: amirbnsl (esi-sba-dz) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


100%|██████████| 170M/170M [00:05<00:00, 29.6MB/s]
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 194MB/s]


Epoch 0: Loss 1.645, Val Acc 53.17%
New best model saved with acc: 53.17
Epoch 1: Loss 1.224, Val Acc 68.57%
New best model saved with acc: 68.57
Epoch 2: Loss 0.892, Val Acc 73.83%
New best model saved with acc: 73.83
Epoch 3: Loss 0.803, Val Acc 75.15%
New best model saved with acc: 75.15
Epoch 4: Loss 0.701, Val Acc 77.60%
New best model saved with acc: 77.6


...

Epoch 0: Loss 1.164, Val Acc 71.89%
New best model saved with acc: 71.89
Epoch 1: Loss 0.779, Val Acc 76.89%
New best model saved with acc: 76.89
Epoch 2: Loss 0.669, Val Acc 78.87%
New best model saved with acc: 78.87
Epoch 3: Loss 0.601, Val Acc 79.89%
New best model saved with acc: 79.89
Epoch 4: Loss 0.547, Val Acc 81.42%
New best model saved with acc: 81.42


batch_loss,█▆▅▅▄▄▄▃▃▄▂▁▄▂▃▁▂▂▁▁▂▁▂▃▂▂▂▂▁▂▂▁▁▂▂▁▁▁▁▁
epoch,▁▃▅▆█
loss,█▄▂▂▁
val_acc,▁▅▆▇█
batch_loss,0.61433
epoch,4
loss,0.54694
val_acc,81.42


wandb: Agent Starting Run: mp6y2eff with config:
wandb: 	batch_size: 128
wandb: 	epochs: 5
wandb: 	learning_rate: 0.00039469230291637653
wandb: 	optimizer: sgd
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.


Epoch 0: Loss 1.551, Val Acc 58.11%
New best model saved with acc: 58.11
Epoch 1: Loss 1.094, Val Acc 66.10%
New best model saved with acc: 66.1
Epoch 2: Loss 0.948, Val Acc 68.86%
New best model saved with acc: 68.86
Epoch 3: Loss 0.869, Val Acc 71.62%
New best model saved with acc: 71.62
Epoch 4: Loss 0.808, Val Acc 73.22%
New best model saved with acc: 73.22


batch_loss,███▅▆▅▄▄▄▄▄▃▃▃▃▃▃▂▃▁▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▃▁▃
epoch,▁▃▅▆█
loss,█▄▂▂▁
val_acc,▁▅▆▇█
batch_loss,0.87027
epoch,4
loss,0.80751
val_acc,73.22
